In [ ]:
import pandas as pd
import toml
import re
import pygsheets
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
from paulssonlab.api.util import base_url
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.commands.parser as cmd_parser
import paulssonlab.cloning.commands.semantics as cmd_semantics

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"])

# SigW storage vectors

In [ ]:
plib_plasmids = reg.get_df(("pLIB", "plasmids"))

In [ ]:
plasmids_to_make = plib_plasmids.loc["pLIB213":"pLIB218"]

In [ ]:
plasmids_to_make["Command"]

In [ ]:
plasmid_maps = {
    name: cmd_semantics.eval_command(cmd, reg.get)["_seq"]
    for name, cmd in plasmids_to_make["Command"].to_dict().items()
}

In [ ]:
workflow.upload_plasmid_maps(
    reg.sheets_client.drive.service, plasmid_maps, reg.registry[("pLIB", "maps")]
)

In [ ]:
reg.get("PsigW")

In [ ]:
plasmid_maps

# PCR debugging

In [ ]:
template = reg.get("pLIB219")["_seq"]
primer1 = reg.get("oLIB189")["_seq"]
primer2 = reg.get("oLIB190")["_seq"]
t = str(template.seq).lower()
p1 = str(primer1).lower()
p2 = str(primer2).lower()

In [ ]:
primer1

In [ ]:
primer2

In [ ]:
p1

In [ ]:
p2

In [ ]:
t_rc = str(sequence.reverse_complement(t))

In [ ]:
sequence.pcr(t, p1, p2, min_score=10)

In [ ]:
_190.seq_lower() == "TGCCGCCGTCTCGGTCTCaAatggaaatgatgattaaaaaaagaattaaacaagtcaaaaaaggcgaccaggatgcatttgcggacatcgtagatatttacaaagataaaatttatcagctttgctaccgtatgcttggcaatgtgcatgaggcggaggatattgcacaggaagctttcatcagagcgtacgttaatatcgacagttttgatattaaccggaaattttcaacttggctttatcgaatcgcgaccaatttgaccattgaccgcattcgcaaaaagaagccggattattacctcgacgcagaggtggctggtacggaaggcttgaccatgtattctcaaatcgtcgcagatggtgttttgcctgaagatgcagttgtatcgctggagctctcaaacacgatacagcagaaaattttaaagcttcctgacaaatacagaacagtcatcgtattaaagtatattgacgaactctcattaattgaaatcggggagattctaaacattcctgtggggactgtgaaaacgcggattcacagaggcagagaggctcttaGAAAACAATTAAGGGATCTTTAAGGTTGAGACCGGAGACGTAACAA".lower()

In [ ]:
len(_)

In [ ]:
t[:10]

In [ ]:
t2 = "gaattcatgagatcttctacaccctgccaaaaatggaaaattttttttcaaaagtaaaattgaaaccttttgaaacgaagctcgtatacatacagaccggtccgcaaggaaggagtgggaatggaaatgatgattaaaaaaagaattaaacaagtcaaaaaaggcgaccaggatgcatttgcggacatcgtagatatttacaaagataaaatttatcagctttgctaccgtatgcttggcaatgtgcatgaggcggaggatattgcacaggaagctttcatcagagcgtacgttaatatcgacagttttgatattaaccggaaattttcaacttggctttatcgaatcgcgaccaatttgaccattgaccgcattcgcaaaaagaagccggattattacctcgacgcagaggtggctggtacggaaggcttgaccatgtattctcaaatcgtcgcagatggtgttttgcctgaagatgcagttgtatcgctggagctctcaaacacgatacagcagaaaattttaaagcttcctgacaaatacagaacagtcatcgtattaaagtatattgacgaactctcattaattgaaatcggggagattctaaacattcctgtggggactgtgaaaacgcggattcacagaggcagagaggctcttagaaaacaattaagggatctttaaggatcctaactcgagtaatgagaagcttgggcccgaacaaaaactcatctcagaagaggatctgaatagcgccgtcgaccatcatcatcatcatcattgagtttaaacggtctccagcttggctgttttggcggatgagagaagattttcagcctgatacagattaaatcagaacgcagaagcggtctgataaaacagaatttgcctggcggcagtagcgcggtggtcccacctgaccccatgccgaactcagaagtgaaacgccgtagcgccgatggtagtgtggggtctccccatgcgagagtagggaactgccaggcatcaaataaaacgaaaggctcagtcgaaagactgggcctttcgttttatctgttgtttgtcggtgaacttctagtctatgctactccatcgagccgtcaattgtctgattcgttaccaattatgacaacttgacggctacatcattcactttttcttcacaaccggcacggaactcgctcgggctggccccggtgcattttttaaatacccgcgagaaatagagttgatcgtcaaaaccaacattgcgaccgacggtggcgataggcatccgggtggtgctcaaaagcagcttcgcctggctgatacgttggtcctcgcgccagcttaagacgctaatccctaactgctggcggaaaagatgtgacagacgcgacggcgacaagcaaacatgctgtgcgacgctggcgatatcaaaattgctgtctgccaggtgatcgctgatgtactgacaagcctcgcgtacccgattatccatcggtggatggagcgactcgttaatcgcttccatgcgccgcagtaacaattgctcaagcagatttatcgccagcagctccgaatagcgcccttccccttgcccggcgttaatgatttgcccaaacaggtcgctgaaatgcggctggtgcgcttcatccgggcgaaagaaccccgtattggcaaatattgacggccagttaagccattcatgccagtaggcgcgcggacgaaagtaaacccactggtgataccattcgcgagcctccggatgacgaccgtagtgatgaatctctcctggcgggaacagcaaaatatcacccggtcggcaaacaaattctcgtccctgatttttcaccaccccctgaccgcgaatggtgagattgagaatataacctttcattcccagcggtcggtcgataaaaaaatcgagataaccgttggcctcaatcggcgttaaacccgccaccagatgggcattaaacgagtatcccggcagcaggggatcattttgcgcttcagccatacttttcatactcccgccattcagagaagaaaccaattgtccatattgcatcagacattgccgtcactgcgtcttttactggctcttctcgctaaccaaaccggtaaccccgcttattaaaagcattctgtaacaaagcgggaccaaagccatgacaaaaacgcgtaacaaaagtgtctataatcacggcagaaaagtccacattgattatttgcacggcgtcacactttgctatgccatagcatttttatccataagattagcggatcttacctgacgctttttatcgcaactctctactgtttctccatacccgctagcggtacctctcccgacccgaaaggaggtaatgatatgagctgtcctgaacaaattgtgcagcttatgcatatgcatcttgatggagatatccttccaaaagatgaacacgtattaaatgaacatctggagacatgcgagaaatgcagaaagcatttttacgagatggagaaatccatagcgctcgtacggagcacatcgcatgtcgaagcccccgcggattttaccgctaatgtcatggcaaaattgcctaaggagaagaaaagagcttctgtaaaaagatggttcagaacccattaataagaaagaggagaaaggatctatggcaagtagcgaagacgttatcaaagagttcatgcgtttcaaagttcgtatggaaggttccgttaacggtcacgagttcgaaatcgaaggtgaaggtgaaggtcgtccgtacgaaggtacccagaccgctaaactgaaagttaccaaaggtggtccgctgccgttcgcttgggacatcctgtccccgcagttccagtacggttccaaagcttacgttaaacacccggctgacatcccggactacctgaaactgtccttcccggaaggtttcaaatgggaacgtgttatgaacttcgaagacggtggtgttgttaccgttacccaggactcctccctgcaagacggtgagttcatctacaaagttaaactgcgtggtaccaacttcccgtccgacggtccggttatgcagaaaaaaaccatgggttgggaagcttccaccgaacgtatgtacccggaagacggtgctctgaaaggtgaaatcaaaatgcgtctgaaactgaaagacggtggtcactacgacgctgaagttaaaaccacctacatggctaaaaaaccggttcagctgccgggtgcttacaaaaccgacatcaaactggacatcacctcccacaacgaagactacaccatcgttgaacagtacgaacgtgctgaaggtcgtcactccaccggtgcttaataacgagctgtacaagtaaggatcctaactcgagtaatgagaagcttgggcccgaacaaaaactcatctcagaagaggatctgaatagcgccgtcgaccatcatcatcatcatcattgagtttaaacggtctccagcttggctgttttggcggatgagagaagattttcagcctgatacagattaaatcagaacgcagaagcggtctgataaaacagaatttgcctggcggcagtagcgcggtggtcccacctgaccccatgccgaactcagaagtgaaacgccgtagcgccgatggtagtgtggggtctccccatgcgagagtagggaactgccaggcatcaaataaaacgaaaggctcagtcgaaagactgggcctttcgttttatctgttgtttgtcggtgaactaatttctagactgcaggcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatgtgagcaaaaggccagcaaaaggccaggaaccgtaaaaaggccgcgttgctggcgtttttccataggctccgcccccctgacgagcatcacaaaaatcgacgctcaagtcagaggtggcgaaacccgacaggactataaagataccaggcgtttccccctggaagctccctcgtgcgctctcctgttccgaccctgccgcttaccggatacctgtccgcctttctcccttcgggaagcgtggcgctttctcatagctcacgctgtaggtatctcagttcggtgtaggtcgttcgctccaagctgggctgtgtgcacgaaccccccgttcagcccgaccgctgcgccttatccggtaactatcgtcttgagtccaacccggtaagacacgacttatcgccactggcagcagccactggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgagattatcaaaaaggatcttcacctagatccttttaaattaaaaatgaagttttaaatcaatctaaagtatatatgagtaaacttggtctgacagttaccaatgcttaatcagtgaggcacctatctcagcgatctgtctatttcgttcatccatagttgcctgactccccgtcgtgtagataactacgatacgggagggcttaccatctggccccagtgctgcaatgataccgcgagacccacgctcaccggctccagatttatcagcaataaaccagccagccggaagggccgagcgcagaagtggtcctgcaactttatccgcctccatccagtctattaattgttgccgggaagctagagtaagtagttcgccagttaatagtttgcgcaacgttgttgccattgctacaggcatcgtggtgtcacgctcgtcgtttggtatggcttcattcagctccggttcccaacgatcaaggcgagttacatgatcccccatgttgtgcaaaaaagcggttagctccttcggtcctccgatcgttgtcagaagtaagttggccgcagtgttatcactcatggttatggcagcactgcataattctcttactgtcatgccatccgtaagatgcttttctgtgactggtgagtactcaaccaagtcattctgagaatagtgtatgcggcgaccgagttgctcttgcccggcgtcaatacgggataataccgcgccacatagcagaactttaaaagtgctcatcattggaaaacgttcttcggggcgaaaactctcaaggatcttaccgctgttgagatccagttcgatgtaacccactcgtgcacccaactgatcttcagcatcttttactttcaccagcgtttctgggtgagcaaaaacaggaaggcaaaatgccgcaaaaaagggaataagggcgacacggaaatgttgaatactcatactcttcctttttcaatattattgaagcatttatcagggttattgtctcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgacgtctaagaaaccattattatcatgacattaacctataaaaataggcgtatcacgaggcagaatttcagataaaaaaaatccttagctttcgctaaggatgatttctg"

In [ ]:
t_rc

In [ ]:
p22 = "ttgttacgtctccggtctcaaccttaaagatcccttaattgttttc"

In [ ]:
sequence.find_primer_binding_site(t2, p22, min_score=10, reverse_complement=True)

In [ ]:
p2

In [ ]:
sequence.find_primer_binding_site(t, p1, min_score=10)

In [ ]:
sequence.find_primer_binding_site(t, p1, min_score=6, try_reverse_complement=True)

In [ ]:
x = sequence.find_primer_binding_site(
    t_rc, p1, min_score=6, try_reverse_complement=True
)
[(-a[0], len(t) - a[1], a[2]) for a in x]

In [ ]:
template[890:900]

In [ ]:
# test pcr circular handling by reindexing across all positions